# Simulating the Limit Order Book

Look at data in df. Aggregate vía time then vía volume, etc and investigate statistical properties.

Loop through data and after each aggregated box, make a decision. Filled according to market orders that arrived and book position (maybe assume that at back of queue first).

For state: store midprice, tick size and your orders on +- 5ticks (start with just the touch)

Aggregate cancellations with limit orders in the incoming block by number. All other cancellations must be ahead of you in the queue and so increase the chance of getting filled.

Question: how do you deal with hidden orders? What are they, and how do they get placed the LOB?

Look at past data and think about percentages of hidden limit orders

In [57]:
import sys

import pandas as pd

from pandas import Timedelta

sys.path.append("../")

from RL4MM.database.PostgresEngine import PostgresEngine

## Retrieve the data

This can be done much more cleanly. Update when possible.

In [12]:
cols = ['id','timestamp', 'type', 'external_id', 'size', 'price', 'direction',
       'ask_price_0', 'ask_size_0', 'bid_price_0', 'bid_size_0', 'ask_price_1',
       'ask_size_1', 'bid_price_1', 'bid_size_1', 'ask_price_2', 'ask_size_2',
       'bid_price_2', 'bid_size_2', 'ask_price_3', 'ask_size_3', 'bid_price_3',
       'bid_size_3', 'ask_price_4', 'ask_size_4', 'bid_price_4', 'bid_size_4',
       'ask_price_5', 'ask_size_5', 'bid_price_5', 'bid_size_5', 'ask_price_6',
       'ask_size_6', 'bid_price_6', 'bid_size_6', 'ask_price_7', 'ask_size_7',
       'bid_price_7', 'bid_size_7', 'ask_price_8', 'ask_size_8', 'bid_price_8',
       'bid_size_8', 'ask_price_9', 'ask_size_9', 'bid_price_9', 'bid_size_9',
       'exchange', 'ticker']

In [7]:
db=PostgresEngine().engine

In [13]:
query = db.engine.execute("SELECT * FROM book_L10 WHERE ticker = 'AAPL'") 
aapl_data = query.fetchall()

aapl_df = pd.DataFrame(aapl_data, columns=cols)
aapl_df.head()

,id,timestamp,type,external_id,size,price,direction,ask_price_0,ask_size_0,bid_price_0,...,ask_price_8,ask_size_8,bid_price_8,bid_size_8,ask_price_9,ask_size_9,bid_price_9,bid_size_9,exchange,ticker
0,NASDAQ_AAPL_2012-06-21_0,2012-06-21 09:30:00.004241,submission,16113575,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5843800.00000000000000000000000000000000,200.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5842700.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
1,NASDAQ_AAPL_2012-06-21_1,2012-06-21 09:30:00.004261,submission,16113584,18.00000000000000000000000000000000,5853200.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5843800.00000000000000000000000000000000,200.00000000000000000000000000000000,NASDAQ,AAPL
2,NASDAQ_AAPL_2012-06-21_2,2012-06-21 09:30:00.004447,submission,16113594,18.00000000000000000000000000000000,5853100.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
3,NASDAQ_AAPL_2012-06-21_3,2012-06-21 09:30:00.025552,submission,16120456,18.00000000000000000000000000000000,5859100.00000000000000000000000000000000,sell,5859100.00000000000000000000000000000000,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,...,5873900.00000000000000000000000000000000,100.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
4,NASDAQ_AAPL_2012-06-21_4,2012-06-21 09:30:00.025580,submission,16120480,18.00000000000000000000000000000000,5859200.00000000000000000000000000000000,sell,5859100.00000000000000000000000000000000,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,...,5871000.00000000000000000000000000000000,10.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5873900.00000000000000000000000000000000,100.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL


In [15]:
aapl_df.drop(columns=["id"],inplace=True)

In [20]:
aapl_df.set_index("timestamp", inplace=True)

In [60]:
latency_ns = 25*10**(3)

In [61]:
latency_ns

25000

In [69]:
now = aapl_df.index[0].round("s")

In [71]:
next_second_activity=aapl_df[aapl_df.index.round("s")==now]

In [72]:
order_arrival_time = now + Timedelta(latency_ns)

In [73]:
order_arrival_time

Timestamp('2012-06-21 09:30:00.000025')

In [74]:
our_order = next_second_activity.iloc[0]

In [75]:
our_order

type                                         submission
external_id                                    16113575
size                18.00000000000000000000000000000000
price          5853300.00000000000000000000000000000000
direction                                           buy
ask_price_0    5859400.00000000000000000000000000000000
ask_size_0         200.00000000000000000000000000000000
bid_price_0    5853300.00000000000000000000000000000000
bid_size_0          18.00000000000000000000000000000000
ask_price_1    5859800.00000000000000000000000000000000
ask_size_1         200.00000000000000000000000000000000
bid_price_1    5853000.00000000000000000000000000000000
bid_size_1         150.00000000000000000000000000000000
ask_price_2    5861000.00000000000000000000000000000000
ask_size_2         200.00000000000000000000000000000000
bid_price_2    5851000.00000000000000000000000000000000
bid_size_2           5.00000000000000000000000000000000
ask_price_3    5868900.0000000000000000000000000

In [64]:
next_second_activity.loc[order_arrival_time]=

,type,external_id,size,price,direction,ask_price_0,ask_size_0,bid_price_0,bid_size_0,ask_price_1,...,ask_price_8,ask_size_8,bid_price_8,bid_size_8,ask_price_9,ask_size_9,bid_price_9,bid_size_9,exchange,ticker
timestamp,,,,,,,,,,,,,,,,,,,,,
2012-06-21 09:30:00.004241,submission,16113575,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859800.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5843800.00000000000000000000000000000000,200.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5842700.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.004261,submission,16113584,18.00000000000000000000000000000000,5853200.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859800.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5843800.00000000000000000000000000000000,200.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.004447,submission,16113594,18.00000000000000000000000000000000,5853100.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859800.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.025552,submission,16120456,18.00000000000000000000000000000000,5859100.00000000000000000000000000000000,sell,5859100.00000000000000000000000000000000,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859400.00000000000000000000000000000000,...,5873900.00000000000000000000000000000000,100.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.025580,submission,16120480,18.00000000000000000000000000000000,5859200.00000000000000000000000000000000,sell,5859100.00000000000000000000000000000000,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859200.00000000000000000000000000000000,...,5871000.00000000000000000000000000000000,10.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5873900.00000000000000000000000000000000,100.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL


In [52]:
for row in next_second_activity.itertuples():
    print(row.direction)

buy
buy
buy
sell
sell
sell
buy
sell
buy
buy
sell
sell
sell
buy
buy
buy
buy
sell
buy
sell
buy
sell
buy
sell
buy
sell
buy
sell
sell
sell
sell
sell
buy
buy
buy
sell
sell
buy
buy
sell
sell
sell
sell
sell
sell
sell
sell
sell
sell
sell
sell
sell
sell
sell
sell
buy
buy
sell
sell
buy
buy
buy
buy
sell
sell
sell
buy
buy
buy
sell
sell
buy
buy
buy
buy
buy
sell
buy
buy
sell
sell


In [48]:
aapl_df.iloc[0].price/10000

Decimal('585.3300000000000000000000000')

In [46]:
aapl_df.head(10)

,type,external_id,size,price,direction,ask_price_0,ask_size_0,bid_price_0,bid_size_0,ask_price_1,...,ask_price_8,ask_size_8,bid_price_8,bid_size_8,ask_price_9,ask_size_9,bid_price_9,bid_size_9,exchange,ticker
timestamp,,,,,,,,,,,,,,,,,,,,,
2012-06-21 09:30:00.004241,submission,16113575,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859800.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5843800.00000000000000000000000000000000,200.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5842700.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.004261,submission,16113584,18.00000000000000000000000000000000,5853200.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859800.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5843800.00000000000000000000000000000000,200.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.004447,submission,16113594,18.00000000000000000000000000000000,5853100.00000000000000000000000000000000,buy,5859400.00000000000000000000000000000000,200.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859800.00000000000000000000000000000000,...,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5879000.00000000000000000000000000000000,500.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.025552,submission,16120456,18.00000000000000000000000000000000,5859100.00000000000000000000000000000000,sell,5859100.00000000000000000000000000000000,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859400.00000000000000000000000000000000,...,5873900.00000000000000000000000000000000,100.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5876500.00000000000000000000000000000000,1160.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.025580,submission,16120480,18.00000000000000000000000000000000,5859200.00000000000000000000000000000000,sell,5859100.00000000000000000000000000000000,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859200.00000000000000000000000000000000,...,5871000.00000000000000000000000000000000,10.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5873900.00000000000000000000000000000000,100.00000000000000000000000000000000,5845300.00000000000000000000000000000000,300.00000000000000000000000000000000,NASDAQ,AAPL
2012-06-21 09:30:00.025613,submission,16120503,18.00000000000000000000000000000000,5859300.00000000000000000000000000000000,sell,5859100.00000000000000000000000000000000,18.00000000000000000000000000000000,5853300.00000000000000000000000000000000,18.00000000000000000000000000000000,5859200.00000000000000000000000000000000,...,5870000.00000000000000000000000000000000,100.00000000000000000000000000000000,5846500.00000000000000000000000000000000,300.00000000000000000000000000000000,5871000.00000

In [36]:
first_time=aapl_df.index[0]

In [39]:
first_time.round("s")

Timestamp('2012-06-21 09:30:00')